# Redes neuronales

In [1]:
# librerías auxiliares
import numpy as np
import pandas as pd
from itertools import *

# modulos de sklearn
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.utils.class_weight import compute_class_weight

# metricas de desempeño
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

Importamos los datasets para cada modelo

In [2]:
data_estimador1 = pd.read_csv("dataset_clasificador_binario.csv")
data_estimador2 = pd.read_csv("dataset_clasificador_multiclase.csv")

## Construcción del modelo para identificar si habrá un fallo en las maquinas

El dataset que se usará para la construcción del modelo es el siguiente:

In [3]:
data_estimador1

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,H,L,M
0,-0.951417,-0.946356,0.067484,0.283054,-1.695647,0,0.0,0.0,1.0
1,-0.901428,-0.878954,-0.729604,0.634238,-1.648511,0,0.0,1.0,0.0
2,-0.951417,-1.013759,-0.227940,0.945286,-1.617087,0,0.0,1.0,0.0
3,-0.901428,-0.946356,-0.590253,-0.048061,-1.585664,0,0.0,1.0,0.0
4,-0.901428,-0.878954,-0.729604,0.002108,-1.554240,0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
9968,-0.601495,-1.081162,0.362908,-1.051442,-1.475680,0,0.0,0.0,1.0
9969,-0.551506,-1.081162,0.518981,-0.820665,-1.428545,0,1.0,0.0,0.0
9970,-0.501517,-0.946356,0.591443,-0.660124,-1.349985,0,0.0,0.0,1.0
9971,-0.501517,-0.878954,-0.729604,0.854982,-1.302849,0,1.0,0.0,0.0


In [4]:
def binary_mlpc(X,Y, num_hidden_layers, num_neurons):
    """ función para realizar experimentos con el MLP
    x: matriz de numpy con caracteristicas
    y: vector numpy con las variables a predecir
    num_hidden_layers: list de enteros con el numero de capdas
        ocultas a usar
    num_neurons: list de enteros con el numero de neuronas a usar
    
    Retorna dataframe con:
        - numero de capas, numero de neuronas
        - recall en el cojunto entrenamiento (promedio de los 5 folds)
        - recall en el cojunto test (promedio de los 5 folds)
        - intervalo de confianza del recall promedio en el cojunto test 
            (desviacion estandar de los 5 folds)

        - auc roc en el cojunto entrenamiento (promedio de los 5 folds)
        - auc roc en el cojunto test (promedio de los 5 folds)
        - intervalo de confianza del auc roc promedio en el cojunto test 
            (desviacion estandar de los 5 folds) 
    """
    #Validamos el modelo
    skf = StratifiedKFold(n_splits=5) 
    resultados = pd.DataFrame()
    idx = 0
    for hidden_layers in num_hidden_layers:
        for neurons in num_neurons:
            print("parametros usados - # capas ocultas: ", hidden_layers, " # neuronas: ", neurons)
            errores_train_recall = []
            errores_test_recall = []

            errores_train_auc = []
            errores_test_auc = []
            
            for j, (train, test) in enumerate(skf.split(X, Y)):
                # para almacenar errores intermedios
                Error = np.zeros(5)
                Xtrain= X[train,:]
                Ytrain = Y[train]
                Xtest = X[test, :]
                Ytest = Y[test]
                
                # Balancear los datos
                rus = RandomUnderSampler(sampling_strategy=0.5)
                Xtrain, Ytrain = rus.fit_resample(Xtrain, Ytrain)
                
                # Definimos y entrenamos el modelo
                hidden_layer_sizes = tuple(hidden_layers*[neurons])
                mlp = MLPClassifier(activation='logistic',solver='sgd', learning_rate_init = 1, learning_rate = 'constant', batch_size=50, hidden_layer_sizes=hidden_layer_sizes, random_state=1,n_iter_no_change =500, max_iter=2000)
                
                # entrenar el MLP
                mlp.fit(X=Xtrain, y=Ytrain)

                # se realizan las predicciones
                y_train_pred = mlp.predict(X=Xtrain)
                y_test_pred = mlp.predict(X=Xtest)


                #Evaluamos las predicciones del modelo con los datos de train y de test
                # calculo del recall
                errores_train_recall.append(recall_score(y_true = Ytrain, y_pred = y_train_pred))
                errores_test_recall.append(recall_score(y_true = Ytest, y_pred = y_test_pred))
                # calculo del area bajo la curva roc
                errores_train_auc.append(roc_auc_score(y_true=Ytrain, y_score=mlp.predict_proba(Xtrain)[:, 1]))
                errores_test_auc.append(roc_auc_score(y_true=Ytest, y_score=mlp.predict_proba(Xtest)[:, 1]))

            print('error para configuracion de params = ' + str(np.mean(Error)) + '+-' + str(np.std(Error)))
        
            resultados.loc[idx,'capas ocultas'] = hidden_layers
            resultados.loc[idx,'neuronas en capas ocultas'] = neurons 
            resultados.loc[idx,'error de entrenamiento (recall)'] = np.mean(errores_train_recall)
            resultados.loc[idx,'error de prueba (recall)'] = np.mean(errores_test_recall)
            resultados.loc[idx,'error de prueba (intervalo de confianza - recall)'] = np.std(errores_test_recall)

            resultados.loc[idx,'error de entrenamiento (auc)'] = np.mean(errores_train_auc)
            resultados.loc[idx,'error de prueba (auc)'] = np.mean(errores_test_auc)
            resultados.loc[idx,'error de prueba (intervalo de confianza - auc)'] = np.std(errores_test_auc)
            idx+=1
    return (resultados)

Realizamos los experimentos para encontrar la mejor combinación de hiperparametros

In [ ]:
# vamos a realizar los experimentos
x_estimador1 = data_estimador1.drop(['Target'], axis=1).values
y_estimador1 = data_estimador1['Target'].values
resultadosMLPC = binary_mlpc(X = x_estimador1, Y=y_estimador1, num_hidden_layers=[1,2,3], num_neurons=[12,16,20])

ver los 5 mejores modelos

In [6]:
resultadosMLPC.sort_values('error de prueba (recall)', ascending=False).head(5)

,capas ocultas,neuronas en capas ocultas,error de entrenamiento (recall),error de prueba (recall),error de prueba (intervalo de confianza - recall),error de entrenamiento (auc),error de prueba (auc),error de prueba (intervalo de confianza - auc)
7,3.0,16.0,0.998485,0.845455,0.094086,0.999996,0.942995,0.018135
4,2.0,16.0,0.999242,0.842424,0.083869,1.000000,0.953251,0.015387
6,3.0,12.0,0.987121,0.827273,0.075210,0.999090,0.935612,0.038220
2,1.0,20.0,1.000000,0.815152,0.096018,1.000000,0.944541,0.020771
5,2.0,20.0,0.999242,0.806061,0.112326,0.999999,0.934226,0.014613


## Construcción del modelo para identificar el tipo de fallo

El dataset que se usará para la construcción del modelo es el siguiente:

In [7]:
data_estimador2

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Failure Type,H,L,M
0,-0.551506,-0.609342,7.369473,-3.549861,0.551157,2.0,0.0,1.0,0.0
1,-0.551506,-0.676745,-0.718456,2.580797,1.305329,2.0,0.0,1.0,0.0
2,-0.601495,-0.744148,-0.467624,0.132547,1.572432,3.0,0.0,1.0,0.0
3,-0.801450,-1.215968,-1.431932,2.079106,1.698127,1.0,0.0,1.0,0.0
4,-0.851439,-1.283370,-0.707308,1.236266,1.729551,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
325,-0.701473,-0.137523,4.080791,-2.385939,1.729551,3.0,0.0,1.0,0.0
326,-0.751461,-0.339731,-1.365044,2.681135,-1.507104,2.0,0.0,1.0,0.0
327,-0.751461,-0.407134,-0.997157,2.099174,1.242481,1.0,0.0,1.0,0.0
328,-0.851439,-0.474537,-1.125360,1.617551,1.541008,1.0,0.0,1.0,0.0


In [8]:
def multiclase_mlpc(X,Y, num_hidden_layers, num_neurons):
    """ función para realizar experimentos con el MLP
    x: matriz de numpy con caracteristicas
    y: vector numpy con las variables a predecir
    num_hidden_layers: list de enteros con el numero de capdas
        ocultas a usar
    num_neurons: list de enteros con el numero de neuronas a usar
    
    Retorna dataframe con:
        - numero de capas, numero de neuronas
        - recall en el cojunto entrenamiento (promedio de los 5 folds)
        - recall en el cojunto test (promedio de los 5 folds)
        - intervalo de confianza del recall promedio en el cojunto test 
            (desviacion estandar de los 5 folds)

        - auc roc en el cojunto entrenamiento (promedio de los 5 folds)
        - auc roc en el cojunto test (promedio de los 5 folds)
        - intervalo de confianza del auc roc promedio en el cojunto test 
            (desviacion estandar de los 5 folds) 
    """
    #Validamos el modelo
    skf = StratifiedKFold(n_splits=5) 
    resultados = pd.DataFrame()
    idx = 0
    for hidden_layers in num_hidden_layers:
        for neurons in num_neurons:
            print("parametros usados - # capas ocultas: ", hidden_layers, " # neuronas: ", neurons)
            errores_train_recall = []
            errores_test_recall = []

            errores_train_auc = []
            errores_test_auc = []
            
            for j, (train, test) in enumerate(skf.split(X, Y)):
                # para almacenar errores intermedios
                Error = np.zeros(5)
                Xtrain= X[train,:]
                Ytrain = Y[train]
                Xtest = X[test, :]
                Ytest = Y[test]
                
                # Balancear los datos
                smote = SMOTE(k_neighbors=3) 
                Xtrain, Ytrain = smote.fit_resample(Xtrain, Ytrain)
                
                # Definimos y entrenamos el modelo
                hidden_layer_sizes = tuple(hidden_layers*[neurons])
                mlp = MLPClassifier(activation='logistic',solver='sgd', learning_rate_init = 1, learning_rate = 'constant', batch_size=50, hidden_layer_sizes=hidden_layer_sizes, random_state=1,n_iter_no_change =500, max_iter=2000)
                
                # se usará la estrategia ovr
                mlp = OneVsRestClassifier(mlp)  

                # entrenar el MLP
                mlp.fit(X=Xtrain, y=Ytrain)

                # se realizan las predicciones
                y_train_pred = mlp.predict(X=Xtrain)
                y_test_pred = mlp.predict(X=Xtest)


                #Evaluamos las predicciones del modelo con los datos de train y de test
                # calculo del recall
                errores_train_recall.append(recall_score(y_true = Ytrain, y_pred = y_train_pred, average='weighted'))
                errores_test_recall.append(recall_score(y_true = Ytest, y_pred = y_test_pred, average='weighted'))
                # calculo del area bajo la curva roc
                errores_train_auc.append(roc_auc_score(y_true=Ytrain, y_score=mlp.predict_proba(Xtrain), multi_class='ovr'))
                errores_test_auc.append(roc_auc_score(y_true=Ytest, y_score=mlp.predict_proba(Xtest), multi_class='ovr'))

            print('error para configuracion de params = ' + str(np.mean(Error)) + '+-' + str(np.std(Error)))
        
            resultados.loc[idx,'capas ocultas'] = hidden_layers
            resultados.loc[idx,'neuronas en capas ocultas'] = neurons 
            resultados.loc[idx,'error de entrenamiento (recall)'] = np.mean(errores_train_recall)
            resultados.loc[idx,'error de prueba (recall)'] = np.mean(errores_test_recall)
            resultados.loc[idx,'error de prueba (intervalo de confianza - recall)'] = np.std(errores_test_recall)

            resultados.loc[idx,'error de entrenamiento (auc)'] = np.mean(errores_train_auc)
            resultados.loc[idx,'error de prueba (auc)'] = np.mean(errores_test_auc)
            resultados.loc[idx,'error de prueba (intervalo de confianza - auc)'] = np.std(errores_test_auc)
            idx+=1
    return (resultados)

Realizamos los experimentos para encontrar la mejor combinación de hiperparametros

In [10]:
# vamos a realizar los experimentos
x_estimador2 = data_estimador2.drop(['Failure Type'], axis=1).values
y_estimador2 = data_estimador2['Failure Type'].values
resultadosMLPC_ovr = multiclase_mlpc(X = x_estimador2, Y=y_estimador2, num_hidden_layers=[1,2,3], num_neurons=[12,16,20])

parametros usados - # capas ocultas:  1  # neuronas:  12
error para configuracion de params = 0.0+-0.0
parametros usados - # capas ocultas:  1  # neuronas:  16
error para configuracion de params = 0.0+-0.0
parametros usados - # capas ocultas:  1  # neuronas:  20
error para configuracion de params = 0.0+-0.0
parametros usados - # capas ocultas:  2  # neuronas:  12
error para configuracion de params = 0.0+-0.0
parametros usados - # capas ocultas:  2  # neuronas:  16
error para configuracion de params = 0.0+-0.0
parametros usados - # capas ocultas:  2  # neuronas:  20
error para configuracion de params = 0.0+-0.0
parametros usados - # capas ocultas:  3  # neuronas:  12
error para configuracion de params = 0.0+-0.0
parametros usados - # capas ocultas:  3  # neuronas:  16
error para configuracion de params = 0.0+-0.0
parametros usados - # capas ocultas:  3  # neuronas:  20
error para configuracion de params = 0.0+-0.0


ver los 5 mejores modelos

In [13]:
resultadosMLPC_ovr.sort_values('error de prueba (recall)', ascending=False).head(5)

,capas ocultas,neuronas en capas ocultas,error de entrenamiento (recall),error de prueba (recall),error de prueba (intervalo de confianza - recall),error de entrenamiento (auc),error de prueba (auc),error de prueba (intervalo de confianza - auc)
2,1.0,20.0,1.000000,0.915152,0.073356,1.000000,0.988198,0.015807
4,2.0,16.0,1.000000,0.912121,0.086986,1.000000,0.985313,0.017126
8,3.0,20.0,1.000000,0.912121,0.079253,1.000000,0.982761,0.018152
0,1.0,12.0,1.000000,0.909091,0.069102,1.000000,0.988479,0.013596
6,3.0,12.0,0.998315,0.909091,0.084632,0.999895,0.984249,0.015056
